In [1]:
import math

from plotly import graph_objects
from plotly.subplots import make_subplots

from airtrack import airplane
from airtrack.discretize import discretize_aoa, discretize_airspeed
from airtrack.solve import solve_altitude

In [9]:
stable_configurations = discretize_aoa(50)

fig = graph_objects.Figure()
fig.add_trace(
    graph_objects.Scatter(
        x=[math.degrees(cfg.aoa) for cfg in stable_configurations],
        y=[math.degrees(cfg.trim) for cfg in stable_configurations],
    )
)
fig.update_layout(
    title_text="Stable AoA Configurations",
    xaxis_title="AoA [deg]",
    yaxis_title="Trim [deg]",
)
fig.show()

In [15]:
MASS = airplane.MAX_MASS * 0.80  # [kg]

# below a certain *critical AoA* there will just not be a stable altitude
# and the airplane will descend until crashing into ground
CRITICAL_AOA = math.radians(0.1)

airspeeds = discretize_airspeed(10)

data = []
for airspeed in airspeeds:
    row = []
    for cfg in stable_configurations:
        if cfg.aoa < CRITICAL_AOA:
            continue
        row.append(solve_altitude(cfg.aoa, cfg.trim, airspeed, MASS))
    data.append(row)

fig = graph_objects.Figure()
fig.add_trace(
    graph_objects.Heatmap(
        y=airspeeds,
        x=[
            cfg.aoa
            for cfg in stable_configurations
            if cfg.aoa >= CRITICAL_AOA
        ],
        z=data,
    )
)
fig.update_layout(
    title_text="Stable Altitudes",
    xaxis_title="AoA [deg]",
    yaxis_title="Airspeed [m/s]",
)
fig.show()